# Instructions

## Please run the cell from top to down 

## The grab_cut windows will pop out and hold right button to drag a rectangle region

## Press 'r' will reset the rectangle

## Press 'n' will stat computing the result

## The result will show in a different window and also in the following cell will have a plot area  that will show the difference between the original image, the mask region and the image after cut

## To use the program more intuitively please run the scripts directly via 

## python2 grabCut_basic.py
## Or
## python2 grabCut_advanced.py
## To run different images..please put the images in the same directory as the script and when running the script type in the file name as the command line argument
## For example 
## python2 grabCut_advanced.py hilary1.jpg


In [6]:
import warnings
import numpy as np
import cv2
import maxflow
import sys
import matplotlib.pyplot as plt
from skimage.color import rgb2grey
from math import exp

In [7]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [8]:
class GrabCut:
    #foreground and background values 
    fore_ground = 1
    back_ground = 0
    #initialize step of the grabCut class
    def __init__(self,img,rec):
        self.img = img
        self.grayImg = rgb2grey(self.img)
        self.num_rows = img.shape[0]
        self.num_cols = img.shape[1]
        #parameters for the energy minimazation
        self.sigma = 30
        self.iteration = 3
        self.energy = np.infty
        self.inif = np.inf
        #the mask same size of the image indicating foreground or background
        #foreground is set to 1, background 0
        #the image layout is 
         #the rect region is set as (x,y,width,height)  
        self.mask = np.zeros(img.shape[:2],np.uint8)
        self.mask[rec[1]:rec[3],rec[0]:rec[2]] = 1
        #initialize the histograms with given mask
        self.foreHistogram,self.backHistogram = self.init_histograms(self.mask)
        #normalize the histograms
        self.normalize_histogram()    
        self.neighbor_xs = [0, 1, 1,  1,  0, -1, -1, -1]
        self.neighbor_ys = [1, 1, 0, -1, -1, -1,  0,  1]
        self.num_neighbors = len(self.neighbor_xs)
        
        #structure to store the N-links
        self.leftW = np.zeros(img.shape[:2],np.float64)
        self.upleftW = np.zeros(img.shape[:2],np.float64)
        self.upW = np.zeros(img.shape[:2],np.float64)
        self.uprightW = np.zeros(img.shape[:2],np.float64)
 
        self.compute()
        self.compute()
        self.compute()
        self.compute()   
        #send to the UI to handle
        testlabelmask2 = self.labelmask
        testlabelmask2 = np.int8(testlabelmask2)
        self.displayMaskImage(testlabelmask2,testlabelmask2)
#        self.displayMaskImage(self.mask,testlabelmask2)
#        
    def compute(self):
        self.foreHistogram,self.backHistogram = self.init_histograms(self.mask)
        self.normalize_histogram() 
        self.compute_labels()
        self.labelmask = self.buildGraph()
        self.mask= self.labelmask
        maskToReturn = np.int8(self.labelmask)
        return maskToReturn
    
    def get_mask(self):  
        return self.mask

    # initialize the histograms with the given mask 
    def init_histograms(self,mask):
        foreHistogram = np.zeros(shape=(16,16,16))
        backHistogram = np.zeros(shape=(16,16,16))
#        print self.img.shape[0] * self.img.shape[1]
        self.foreGroundPixels = np.transpose(np.where(mask==1))
#        print foreGroundPixels.shape
        self.backGroundPixels = np.transpose(np.where(mask==0))
#        print backGroundPixels.shape
        for i in self.foreGroundPixels:
            pixelColor = self.img[i[0],i[1]]
            r = np.rint(np.floor(pixelColor[0]/16))
            g = np.rint(np.floor(pixelColor[1]/16))
            b = np.rint(np.floor(pixelColor[2]/16))
            foreHistogram[r,g,b] = foreHistogram[r,g,b] + 1
        for j in self.backGroundPixels:
            pixelColor2 = self.img[j[0],j[1]]
            r2 = np.rint(np.floor(pixelColor2[0]/16))
            g2 = np.rint(np.floor(pixelColor2[1]/16))
            b2 = np.rint(np.floor(pixelColor2[2]/16))
            backHistogram[r2,g2,b2] = backHistogram[r2,g2,b2] + 1
        return foreHistogram,backHistogram
    
    #normalize the histogram
    def normalize_histogram(self):
        # not do this...way....class...
        self.foreHistogram = self.foreHistogram / np.count_nonzero(self.mask)
        #number of the pixels in the background histogram would just be the total number of pixels - foreGround count
        self.backHistogram = self.backHistogram / ((self.num_rows * self.num_cols) - np.count_nonzero(self.mask))
        
    def compute_labels(self):
        for i in self.foreGroundPixels:
            pixelColor = self.img[i[0],i[1]]
            r = np.rint(np.floor(pixelColor[0]/16))
            g = np.rint(np.floor(pixelColor[0]/16))
            b = np.rint(np.floor(pixelColor[0]/16))
#           case where the pixel appears with a higher chance in the foreground histogram model
            if self.foreHistogram[r,g,b] < self.backHistogram[r,g,b]:
                self.mask[i[0],i[1]] = 0
    def buildGraph(self):
#create the graph using the maxflow libaries 
        graph = maxflow.Graph[float](self.num_rows, self.num_cols)
        nodes = graph.add_grid_nodes((self.num_rows, self.num_cols))
 #        #assign T links to the clearly background pixels
        for j in self.backGroundPixels:
            graph.add_tedge(nodes[j[0],j[1]],self.inif,0)
#        for j in self.backGroundPixels:
        for i in self.foreGroundPixels:
            pixelColor = self.img[i[0],i[1]]
        ## adding in the t-links
            p_f = -np.log(self.foreHistogram[np.floor(pixelColor[0]/16),np.floor(pixelColor[1]/16),np.floor(pixelColor[2]/16)])
            p_b = -np.log(self.backHistogram[np.floor(pixelColor[0]/16),np.floor(pixelColor[1]/16),np.floor(pixelColor[2]/16)])
            graph.add_tedge(nodes[i[0],i[1]],p_f,p_b)
        ##assign N links  
        for x in range(self.num_rows):  
            for y in range(self.num_cols):
                for k in xrange(self.num_neighbors):
                    nx = x + self.neighbor_xs[k]
                    ny = y + self.neighbor_ys[k]
                    #test to see if the the pixal point is out of the picture
                    if nx < 0 or ny < 0 or nx >= self.num_rows or ny >= self.num_cols:
                        continue 
                    weight_n = exp(-((self.grayImg[nx,ny] - self.grayImg[x,y])/(2*self.sigma**2)))
                    #add in the n-links
                    graph.add_edge(nodes[x,y], nodes[nx,ny], weight_n, weight_n)
        print 'the current energy is'
        print graph.maxflow()
        label_mask = graph.get_grid_segments(nodes)
          ##return the result..
        return label_mask
        
        #only compute it once can increase performance a lot!
#    def computeNlinks(self):
        
        
    def displayMaskImage(self,mask,labelmask):
        img = self.img
        masked_img = cv2.bitwise_and(img,img,mask = labelmask)
#        masked_img2 = cv2.bitwise_and(img,img,mask = labelmask)
        plt.subplot(221)
        plt.imshow(img)
        plt.subplot(222)
        plt.imshow(mask,'gray')
        plt.subplot(223)
        plt.imshow(masked_img)
#        plt.subplot(224)
#        plt.imshow(masked_img2)
        plt.show()

In [9]:
#UI interface 
BLUE = [255,0,0]        # rectangle color
RED = [0,0,255]         # PR BG
GREEN = [0,255,0]       # PR FG
BLACK = [0,0,0]         # sure BG
WHITE = [255,255,255]   # sure FG

DRAW_BG = {'color' : BLACK, 'val' : 0}
DRAW_FG = {'color' : WHITE, 'val' : 1}
DRAW_PR_FG = {'color' : GREEN, 'val' : 3}
DRAW_PR_BG = {'color' : RED, 'val' : 2}

# setting up flags
rect = (0,0,1,1)
drawing = False         # flag for drawing curves
rectangle = False       # flag for drawing rect
rect_over = False       # flag to check if rect drawn
rect_or_mask = 100      # flag for selecting rect or mask mode
value = DRAW_FG         # drawing initialized to FG
thickness = 3           # brush thickness

def onmouse(event,x,y,flags,param):
    global img,img2,drawing,value,mask,rectangle,rect,rect_or_mask,ix,iy,rect_over

    # Draw Rectangle
    if event == cv2.EVENT_RBUTTONDOWN:
        rectangle = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if rectangle == True:
            img = img2.copy()
            cv2.rectangle(img,(ix,iy),(x,y),BLUE,2)
            rect = (min(ix,x),min(iy,y),abs(ix-x),abs(iy-y))
            rect_or_mask = 0

    elif event == cv2.EVENT_RBUTTONUP:
        rectangle = False
        rect_over = True
        cv2.rectangle(img,(ix,iy),(x,y),BLUE,2)
        rect = (min(ix,x),min(iy,y),abs(ix-x),abs(iy-y))
        rect_or_mask = 0
        print(" Now press the key 'n' a few times until no further change \n")

    # draw touchup curves

    if event == cv2.EVENT_LBUTTONDOWN:
        if rect_over == False:
            print("first draw rectangle \n")
        else:
            drawing = True
            cv2.circle(img,(x,y),thickness,value['color'],-1)
            cv2.circle(mask,(x,y),thickness,value['val'],-1)

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            cv2.circle(img,(x,y),thickness,value['color'],-1)
            cv2.circle(mask,(x,y),thickness,value['val'],-1)

    elif event == cv2.EVENT_LBUTTONUP:
        if drawing == True:
            drawing = False
            cv2.circle(img,(x,y),thickness,value['color'],-1)
            cv2.circle(mask,(x,y),thickness,value['val'],-1)

if __name__ == '__main__':

    # print documentation
    print(__doc__)

    # Loading images
    if len(sys.argv) == 2:
        filename = sys.argv[1] # for drawing purposes
    else:
        print("No input image given, so loading default image, ../data/lena.jpg \n")
        print("Correct Usage: python grabcut.py <filename> \n")
        filename = 'rose.bmp'

    img = cv2.imread(filename)
    img2 = img.copy()                               # a copy of original image
    mask = np.zeros(img.shape[:2],dtype = np.uint8) # mask initialized to PR_BG
    output = np.zeros(img.shape,np.uint8)           # output image to be shown

    # input and output windows
    cv2.namedWindow('output')
    cv2.namedWindow('input')
    cv2.setMouseCallback('input',onmouse)
    cv2.moveWindow('input',img.shape[1]+10,90)

    print(" Instructions: \n")
    print(" Draw a rectangle around the object using right mouse button \n")

    while(1):

        cv2.imshow('output',output)
        cv2.imshow('input',img)
        k = 0xFF & cv2.waitKey(1)

        # key bindings
        if k == 27:         # esc to exit
            break
        elif k == ord('0'): # BG drawing
            print(" mark background regions with left mouse button \n")
            value = DRAW_BG
        elif k == ord('1'): # FG drawing
            print(" mark foreground regions with left mouse button \n")
            value = DRAW_FG
        elif k == ord('2'): # PR_BG drawing
            value = DRAW_PR_BG
        elif k == ord('3'): # PR_FG drawing
            value = DRAW_PR_FG
        elif k == ord('s'): # save image
            bar = np.zeros((img.shape[0],5,3),np.uint8)
            res = np.hstack((img2,bar,img,bar,output))
            cv2.imwrite('grabcut_output.png',res)
            print(" Result saved as image \n")
        elif k == ord('r'): # reset everything
            print("resetting \n")
            rect = (0,0,1,1)
            drawing = False
            rectangle = False
            rect_or_mask = 100
            rect_over = False
            value = DRAW_FG
            img = img2.copy()
            mask = np.zeros(img.shape[:2],dtype = np.uint8) # mask initialized to PR_BG
            output = np.zeros(img.shape,np.uint8)           # output image to be shown
        elif k == ord('n'): # segment the image
            print(""" For finer touchups, mark foreground and background after pressing keys 0-3
            and again press 'n' \n""")
            myGrabCut = GrabCut(img,rect)
            mask = myGrabCut.compute()
        output = cv2.bitwise_and(img,img,mask=mask)

    cv2.destroyAllWindows()


Automatically created module for IPython interactive environment
No input image given, so loading default image, ../data/lena.jpg 

Correct Usage: python grabcut.py <filename> 

 Instructions: 

 Draw a rectangle around the object using right mouse button 



# Reflection

### The above implementation of grab-cut is a very basic one. It doesn’t run very fast and has lots room for the speed improvement. For the N-links weight, it uses the same edge weight function from the previous graph cut assignment, which is using the intensity of the grayscale of the pixel difference as well as a hard coded sigma values to decide the weight of the edge. Because of this, this method fails really bad when there are some pixels inside the foreground region that appears to be much more similar to the background regions. And this will result in the leak in of black pixels.

### For the running speed of this naive implementation, because it actually calculates the N-links every time when it does an iteration,  this method does too many unnecessary calculations. And that’s why it is not very fast

### Both of the issues stated above have been attempted to fix in the method below.

## The following is a an enhancement of the basic grabCut from the above

### This one uses a different weight function for the N-links
### lamda / distance(p,q) x e -(- distance(p_color,q_color))/ 2* sigma^2
### It uses the pixel color differences instead of the grayscale intensity difference 
### taken into account the distance between the edge as well.
### also introduces a new parameter lamda and still uses the sigma

## To increase the speed
### It computes the N-links weight only once and store that into 4 different arrays the same as the image
### when constructing the graph, it will be able to access the weight directly, which saves some computing time 

### The issues with this implementation is that because it uses the color distance as well as the pixel distance. Around the edge of the foreground object, it appears to have very bad smoothness,

In [10]:
import numpy as np
import warnings
import cv2
import maxflow
import sys
import matplotlib.pyplot as plt
from skimage.color import rgb2grey

In [11]:
class GrabCut2:
    #foreground and background values 
    fore_ground = 1
    back_ground = 0
    #initialize step of the grabCut class
    def __init__(self,img,rec):
        self.img = img
        self.grayImg = rgb2grey(self.img)
        self.num_rows = img.shape[1]
        self.num_cols = img.shape[0]
        #parameters for the energy minimazation
        self.lame = 5
        self.sigma = 10
        self.iteration = 3
        self.energy = np.infty
        self.inif = np.inf
        #the mask same size of the image indicating foreground or background
        #foreground is set to 1, background 0
        #the image layout is 
         #the rect region is set as (x,y,width,height)  
        self.mask = np.zeros(img.shape[:2],np.uint8)
        self.mask[rec[1]:rec[3],rec[0]:rec[2]] = 1
        #initialize the histograms with given mask
        self.foreHistogram,self.backHistogram = self.init_histograms(self.mask)
        #normalize the histograms
        self.normalize_histogram()    
        self.neighbor_xs = [0, 1, 1,  1,  0, -1, -1, -1]
        self.neighbor_ys = [1, 1, 0, -1, -1, -1,  0,  1]
        self.num_neighbors = len(self.neighbor_xs)
        
        #structure to store the N-links
        self.rightW = np.zeros(img.shape[:2],np.float64)
        self.upleftW = np.zeros(img.shape[:2],np.float64)
        self.upW = np.zeros(img.shape[:2],np.float64)
        self.uprightW = np.zeros(img.shape[:2],np.float64)
        #compute it once and use it all the time
        self.computeNlinks()
        self.compute()
        self.compute()
        self.compute()
        self.compute() 
        #send to the UI to handle
        testlabelmask2 = self.labelmask
        testlabelmask2 = np.int8(testlabelmask2)
#        self.displayMaskImage(self.mask,testlabelmask2)
#        
    def compute(self):
        self.foreHistogram,self.backHistogram = self.init_histograms(self.mask)
        self.normalize_histogram() 
        self.compute_labels()
        self.labelmask = self.buildGraph()
        self.mask= self.labelmask
        maskToReturn = np.int8(self.labelmask)
        return maskToReturn
    
    def get_mask(self):  
        return self.mask

    # initialize the histograms with the given mask 
    def init_histograms(self,mask):
        foreHistogram = np.zeros(shape=(16,16,16))
        backHistogram = np.zeros(shape=(16,16,16))
#        print self.img.shape[0] * self.img.shape[1]
        self.foreGroundPixels = np.transpose(np.where(mask==1))
#        print foreGroundPixels.shape
        self.backGroundPixels = np.transpose(np.where(mask==0))
#        print backGroundPixels.shape
        for i in self.foreGroundPixels:
            pixelColor = self.img[i[0],i[1]]
            r = np.rint(np.floor(pixelColor[0]/16))
            g = np.rint(np.floor(pixelColor[1]/16))
            b = np.rint(np.floor(pixelColor[2]/16))
            foreHistogram[r,g,b] = foreHistogram[r,g,b] + 1
        for j in self.backGroundPixels:
            pixelColor2 = self.img[j[0],j[1]]
            r2 = np.rint(np.floor(pixelColor2[0]/16))
            g2 = np.rint(np.floor(pixelColor2[1]/16))
            b2 = np.rint(np.floor(pixelColor2[2]/16))
            backHistogram[r2,g2,b2] = backHistogram[r2,g2,b2] + 1
        return foreHistogram,backHistogram
    
    #normalize the histogram
    def normalize_histogram(self):
        # not do this...way....class...
        self.foreHistogram = self.foreHistogram / np.count_nonzero(self.mask)
        #number of the pixels in the background histogram would just be the total number of pixels - foreGround count
        self.backHistogram = self.backHistogram / ((self.num_rows * self.num_cols) - np.count_nonzero(self.mask))
        
    def compute_labels(self):
        for i in self.foreGroundPixels:
            pixelColor = self.img[i[0],i[1]]
            r = np.rint(np.floor(pixelColor[0]/16))
            g = np.rint(np.floor(pixelColor[0]/16))
            b = np.rint(np.floor(pixelColor[0]/16))
#           case where the pixel appears with a higher chance in the foreground histogram model
            if self.foreHistogram[r,g,b] < self.backHistogram[r,g,b]:
                self.mask[i[0],i[1]] = 0
    def buildGraph(self):
        
#create the graph using the maxflow libaries 
        graph = maxflow.Graph[float](self.num_cols, self.num_rows)
        nodes = graph.add_grid_nodes((self.num_cols, self.num_rows))
 #        #assign T links to the clearly background pixels
        for j in self.backGroundPixels:
            graph.add_tedge(nodes[j[0],j[1]],self.inif,0)
#        for j in self.backGroundPixels:
        for i in self.foreGroundPixels:
            pixelColor = self.img[i[0],i[1]]
        ## adding in the t-links
            p_f = -np.log(self.foreHistogram[np.floor(pixelColor[0]/16),np.floor(pixelColor[1]/16),np.floor(pixelColor[2]/16)])
            p_b = -np.log(self.backHistogram[np.floor(pixelColor[0]/16),np.floor(pixelColor[1]/16),np.floor(pixelColor[2]/16)])
            graph.add_tedge(nodes[i[0],i[1]],p_f,p_b)
        ##assign N links  
        for y in range(self.num_rows):  
            for x in range(self.num_cols):
                if x > 0 and y < (self.num_rows -1):
                    graph.add_edge(nodes[x,y],nodes[x-1,y+1],self.upleftW[x,y],self.upleftW[x,y])
                if y < (self.num_rows -1):
                    graph.add_edge(nodes[x,y],nodes[x,y+1],self.upW[x,y],self.upW[x,y])
                if (x < (self.num_cols -1)) and (y < (self.num_rows -1)):
                    graph.add_edge(nodes[x,y],nodes[x+1,y+1],self.uprightW[x,y],self.uprightW[x,y])
                if x < (self.num_cols - 1):
                    graph.add_edge(nodes[x,y],nodes[x+1,y],self.rightW[x,y],self.rightW[x,y])
        print 'the current energy is'
        print graph.maxflow()
        label_mask = graph.get_grid_segments(nodes)
          ##return the result..
        return label_mask
        
#        only compute it once can increase performance a lot!
    def computeNlinks(self):
        lampSqrt = self.lame/ np.sqrt(2)
        for y in range(self.num_rows-1):
            for x in range(self.num_cols-1):
#                print (x,y)
                if x > 0 and (y < (self.num_rows - 1)):
                    self.upleftW[x,y] =  lampSqrt * np.exp(-((self.distance(self.img[x,y],self.img[x-1,y+1]))/(2*self.sigma**2)))
                if y < (self.num_rows - 1):
                    self.upW[x,y] = self.lame * np.exp(-((self.distance(self.img[x,y],self.img[x,y+1]))/(2*self.sigma**2)))
                if (x < (self.num_cols - 1)) and (y < (self.num_rows - 1)):
                    self.uprightW[x,y] = lampSqrt * np.exp(-((self.distance(self.img[x,y],self.img[x+1,y+1]))/(2*self.sigma**2)))
                if x < (self.num_cols - 1):
                    self.rightW[x,y] = self.lame * np.exp(-((self.distance(self.img[x,y],self.img[x+1,y]))/(2*self.sigma**2)))
    
    def computeN(self,x,y,x1,y1):
         return np.exp(-((self.distance(self.img[x,y],self.img[x1,y1]))/(2*self.sigma**2)))
         
    def distance(self,x,y):
        return np.linalg.norm(x-y)

    def displayMaskImage(self,mask,labelmask):
        img = self.img
        masked_img = cv2.bitwise_and(img,img,mask = labelmask)
#        masked_img2 = cv2.bitwise_and(img,img,mask = labelmask)
        plt.subplot(221)
        plt.imshow(img)
        plt.subplot(222)
        plt.imshow(mask,'gray')
        plt.subplot(223)
        plt.imshow(masked_img)
        plt.show()

In [12]:
BLUE = [255,0,0]        # rectangle color
RED = [0,0,255]         # PR BG
GREEN = [0,255,0]       # PR FG
BLACK = [0,0,0]         # sure BG
WHITE = [255,255,255]   # sure FG

DRAW_BG = {'color' : BLACK, 'val' : 0}
DRAW_FG = {'color' : WHITE, 'val' : 1}
DRAW_PR_FG = {'color' : GREEN, 'val' : 3}
DRAW_PR_BG = {'color' : RED, 'val' : 2}

# setting up flags
rect = (0,0,1,1)
drawing = False         # flag for drawing curves
rectangle = False       # flag for drawing rect
rect_over = False       # flag to check if rect drawn
rect_or_mask = 100      # flag for selecting rect or mask mode
value = DRAW_FG         # drawing initialized to FG
thickness = 3           # brush thickness

def onmouse(event,x,y,flags,param):
    global img,img2,drawing,value,mask,rectangle,rect,rect_or_mask,ix,iy,rect_over

    # Draw Rectangle
    if event == cv2.EVENT_RBUTTONDOWN:
        rectangle = True
        ix,iy = x,y

    elif event == cv2.EVENT_MOUSEMOVE:
        if rectangle == True:
            img = img2.copy()
            cv2.rectangle(img,(ix,iy),(x,y),BLUE,2)
            rect = (min(ix,x),min(iy,y),abs(ix-x),abs(iy-y))
            rect_or_mask = 0

    elif event == cv2.EVENT_RBUTTONUP:
        rectangle = False
        rect_over = True
        cv2.rectangle(img,(ix,iy),(x,y),BLUE,2)
        rect = (min(ix,x),min(iy,y),abs(ix-x),abs(iy-y))
        rect_or_mask = 0
        print(" Now press the key 'n' a few times until no further change \n")

    # draw touchup curves

    if event == cv2.EVENT_LBUTTONDOWN:
        if rect_over == False:
            print("first draw rectangle \n")
        else:
            drawing = True
            cv2.circle(img,(x,y),thickness,value['color'],-1)
            cv2.circle(mask,(x,y),thickness,value['val'],-1)

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            cv2.circle(img,(x,y),thickness,value['color'],-1)
            cv2.circle(mask,(x,y),thickness,value['val'],-1)

    elif event == cv2.EVENT_LBUTTONUP:
        if drawing == True:
            drawing = False
            cv2.circle(img,(x,y),thickness,value['color'],-1)
            cv2.circle(mask,(x,y),thickness,value['val'],-1)

if __name__ == '__main__':

    # print documentation
    print(__doc__)

    # Loading images
    if len(sys.argv) == 2:
        filename = sys.argv[1] # for drawing purposes
    else:
        print("No input image given, so loading default image, ../data/lena.jpg \n")
        print("Correct Usage: python grabcut.py <filename> \n")
        filename = 'rose.bmp'

    img = cv2.imread(filename)
    img2 = img.copy()                               # a copy of original image
    mask = np.zeros(img.shape[:2],dtype = np.uint8) # mask initialized to PR_BG
    output = np.zeros(img.shape,np.uint8)           # output image to be shown

    # input and output windows
    cv2.namedWindow('output')
    cv2.namedWindow('input')
    cv2.setMouseCallback('input',onmouse)
    cv2.moveWindow('input',img.shape[1]+10,90)

    print(" Instructions: \n")
    print(" Draw a rectangle around the object using right mouse button \n")

    while(1):

        cv2.imshow('output',output)
        cv2.imshow('input',img)
        k = 0xFF & cv2.waitKey(1)

        # key bindings
        if k == 27:         # esc to exit
            break
        elif k == ord('0'): # BG drawing
            print(" mark background regions with left mouse button \n")
            value = DRAW_BG
        elif k == ord('1'): # FG drawing
            print(" mark foreground regions with left mouse button \n")
            value = DRAW_FG
        elif k == ord('2'): # PR_BG drawing
            value = DRAW_PR_BG
        elif k == ord('3'): # PR_FG drawing
            value = DRAW_PR_FG
        elif k == ord('s'): # save image
            bar = np.zeros((img.shape[0],5,3),np.uint8)
            res = np.hstack((img2,bar,img,bar,output))
            cv2.imwrite('grabcut_output.png',res)
            print(" Result saved as image \n")
        elif k == ord('r'): # reset everything
            print("resetting \n")
            rect = (0,0,1,1)
            drawing = False
            rectangle = False
            rect_or_mask = 100
            rect_over = False
            value = DRAW_FG
            img = img2.copy()
            mask = np.zeros(img.shape[:2],dtype = np.uint8) # mask initialized to PR_BG
            output = np.zeros(img.shape,np.uint8)           # output image to be shown
        elif k == ord('n'): # segment the image
            print(""" For finer touchups, mark foreground and background after pressing keys 0-3
            and again press 'n' \n""")
            myGrabCut = GrabCut2(img,rect)
            mask = myGrabCut.compute()
        output = cv2.bitwise_and(img,img,mask=mask)

    cv2.destroyAllWindows()


Automatically created module for IPython interactive environment
No input image given, so loading default image, ../data/lena.jpg 

Correct Usage: python grabcut.py <filename> 

 Instructions: 

 Draw a rectangle around the object using right mouse button 



## Future improvements

### Due to unsatisfying planning  of timing there are a lot to be improved with this very basic prototype

### 1. Organize the code better to have a better architecture. For example, make the histogram functions into a separate class like most smart people would do.

### 2. Implement the refine methods of the grab-cut, so that the user can edit in the new hard constraints of the foreground and background pixels

### 3. Try to implement the boarder Matting, so the area around the edges will appear to be more natural and smooth.



### 4.Use Gaussian Mixture model for the model estimation 

### 5.Find ways to decide better with the hard code parameters like sigma.

### 6.Mix the histogram method combine with uniform distribution so that it won’t be as discreet.

### 7.Suggest a way to automatically select the initial hard constraints(the user’s input rectangle) 
